# eye center model

In [1]:
import sys
import math 
import warnings
sqrt = math.sqrt

def load_modules():
    sys.path.append("../../../")
    %run ../../../ac/visualizer/plotter.py
    %run ../../../ac/images/filters/filters.py
    %run ../../../ac/common/nps.py
    %run ../../../al/maths/angles.py
    %run ../../../al/maths/linalg/rotations.py
    %run ../../../al/actor/camera.py
    %run ../../../al/optics/rays/plane.py
    %run ../../../al/model/eye/eyeball.py
    %run ../../../al/model/gaze/item.py
    %run ../../../ds/device/nexus5x.py
    warnings.filterwarnings("ignore")
    np.set_printoptions(precision=6, suppress=True)

load_modules()
this_path = "/home/chy/dev-bench/auto-catch/notes/research/eye-3d-center/"

def get_img(src, filename):
    path = "{}/{}/{}.jpg".format(this_path, src, filename)
    img = cv2.imread(path)
    img = bgr_to_rgb(img)
    return img

# real scenario

<br>

#### - 이미지 좌우 반전 여부를 카메라 좌표계 기준으로 확인 할 것!

<br>

## 이미지 파일명
#### - A: 렌즈 정면
#### - B: P 포인트, Nexus 스크린 우상단 지점 (56, 9, 0)
#### - P1: 캘리브레이션 1
#### - P2: 캘리브레이션 2

<br>

## 좌표 지점명
#### - O: 카메라 원점
#### - E: 안구 3차원 중심
#### - I: P 포인트 응시 이미지에서 IRIS CENTER NORMAL VEC
#### - P: P 포인트 

In [2]:
img_a = get_img("center", "A")
img_b = get_img("center", "B") 
img_p1 = get_img("center", "P1-EYE")
img_p2 = get_img("center", "P2-EYE")

img_ca = get_img("center-30", "A")
img_cb = get_img("center-30", "B")
img_cc = get_img("center-30", "C")
img_cd = get_img("center-30", "D")
img_ce = get_img("center-30", "E")

# show_image(img_p1, fig_size=(4, 4))
# show_image(img_p2, fig_size=(4, 4))

# compute rough eye-ball (center-30 profile)


| P | px | py | x | y | comment |
|---|---|---|---|---|---|
| A | 342 | 627 | 0  |  0   | 렌즈 |
| B | 330 | 624 | -55 | 9   | 우상 |
| C | 330 | 627 | -55 | 68  | 우중 |
| D | 330 | 632 | -55 | 125 | 우하 |
|---|---|---|---|---|---|
| E | 333 | 632 | -23 | 125 | 중하 |
| F | 333 | 627 | -23 | 68  | 중중 |
| G | 333 | 619 | -23 | 9   | 중상 |
|---|---|---|---|---|---|
| H | 336 | 619 | 10  | 9   | 좌상 |
| I | 336 | 623 | 10  | 68  | 좌중 |
| J | 336 | 632 | 10  | 125 | 좌하 |


In [3]:
cam = Camera.from_nexus_5x()
GazeItem.set_camera(cam)

In [4]:
def g_item(name, camera=cam):
    cali_points = {
        "A" : (342, 627,  0,    0),
        "B" : (330, 619, -55,   9),
        "C" : (330, 627, -55,  68),
        "D" : (330, 632, -55, 125),
        "E" : (333, 632, -23, 125),
        "F" : (333, 627, -23,  68),
        "G" : (333, 619, -23,   9),
        "H" : (336, 619, 10,    9),
        "I" : (336, 623, 10,   68),
        "J" : (336, 632, 10,  125),
    }
    
    px, py, tx, ty = cali_points[name.upper()]
    return GazeItem(tx, ty, px, py)

In [5]:
ccs = {}
for c in "abcdefghij":
     ccs[c] = g_item(c)

In [6]:
eye = EyeballModel()

for c in "bcdhj":
    eye.add_calibration(ccs[c])
    
eye.optimize_eyeball_params()

      fun: 263.1864059229083
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.013756, -0.004786,  0.006901])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 92
      nit: 17
   status: 0
  success: True
        x: array([  3.232285,   1.854559, 318.752601])


# compute rough eye-ball (c20 profile)


| P | px | py | x | y | comment |
|---|---|---|---|---|---|
| 0 |-| 좌상 |
| 1 |-| 중상 |
| 2 |-| 우상 |
| 3 |-| 우중 |
| 4 |-| 우하 |
| 5 |-| 중하 |
| 6 |-| 좌하 |
| 7 |-| 좌중 |
| 8 |-| 좌상 |
| 9 |-| 중중 |

# Notice

#### [125mm 기준] 안경 (8 ~ 13px) / NO 안경 (19 ~ 27px)
#### 이 기준으로는 가장 최적이 4.5 mm 한계로 보임
#### 현재 이미지는 렌즈 중점에 한정됨

In [7]:
def g_item(idx):
    cali_points = {
        0 : (445, 650,  10,   9),
        1 : (440, 650, -23,   9),
        2 : (430, 651, -55,   9),
        3 : (432, 660, -55,  68),
        4 : (432, 672, -55, 125),
        5 : (440, 670, -23, 125),
        6 : (458, 670,  10, 125),
        7 : (465, 658,  10,  68),
        8 : (466, 643,  10,   9),
        9 : (458, 647, -23, 68),
    }
    
    px, py, tx, ty = cali_points[idx]
    return GazeItem(tx, ty, px, py)

In [8]:
eye = EyeballModel()

ccs = {}
for c in range(0, 10):
    ccs[c] = g_item(c)
for c in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    eye.add_calibration(ccs[c])
    
eye.optimize_eyeball_params()

      fun: 1991652133
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([0., 0., 0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 4
      nit: 0
   status: 0
  success: True
        x: array([  0.,   0., 200.])


# 최적화 검증

In [9]:
E, r = np.array([3.016846,   1.601401, 299.095428]), 12.020592
eye.compute_optical_error(E, r)

1991652133

In [10]:
# eye.estimate_gaze_point(g_item("i").get_oc_vec())
g_item(0).evaluate(eye)

*** NORM: 14128.700612582885 ***


# eyeball sphere center mock scenario

<br>

### 2차원 상의 가상 시나리오 
- www.desmos.com/calculator/6j8jcbmpa2
- OE 60mm, z(OB) 51mm / x(OB) -9mm, P = -60mm

In [11]:
# cam = Camera.from_nexus_5x()
# eye = EyeballModel()

# # OE 60mm, z(OB) 51mm / x(OB) -9mm, P = -60mm
# vec_oe = np.array([0, 0, 1])
# vec_oi = np.array([9/sqrt(2581), 0, 50/sqrt(2581)])
# vec_op = np.array([60, 0, 0])

# eye.set_center_vector(vec_oe, vec_oi, vec_op)
# eye.stage1()

# eyeball sphere iris center mock scenario

<br>

- E : 안구 중심 3d pos
- ER : 안구 반지름
- C11, C12 : 홍채 경계 점1, 점2
- C : 홍채 중심 3d pos (depth 제외)
- CR : 홍채 disk 반지름

In [12]:
# cos = lambda x: np.cos(np.deg2rad(x))
# sin = lambda x: np.sin(np.deg2rad(x))

# ER = 13
# E = (0, 0, 60)    

# c11 = np.array([ER*cos(230), ER*sin(230)])
# c12 = np.array([ER*cos(280), ER*sin(280)])
# C = (c11 + c12) / 2
# CR = np.linalg.norm(c11 - c12) / 2

# print(E, ER, c11, c12, C, CR)
# eye.estimate_iris_center_depth(E, ER, C, CR)